## 

## Проверяем приколы в данных

In [1]:
import pandas as pd

data = pd.read_csv("recipes_normalized.csv")
# norm_df
data

,url,name,ingredients,ingredients_normalized
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,"{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах...","{'Молоко': '250 мл', 'Клубника': '200 г', 'Сах..."
1,https://www.povarenok.ru/recipes/show/1306/,Рулетики,"{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку...","{'Сыр твердый': None, 'Чеснок': None, 'Яйцо ку..."
2,https://www.povarenok.ru/recipes/show/10625/,"Салат ""Баклажанчик""","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '...","{'Баклажан': '3 шт', 'Лук репчатый': '2 шт', '..."
3,https://www.povarenok.ru/recipes/show/167337/,Куриные котлеты с картофельным пюре в духовке,"{'Фарш куриный': '800 г', 'Пюре картофельное':...","{'Фарш куриный': '800 г', 'Картофель': '1 кг',..."
4,https://www.povarenok.ru/recipes/show/91919/,Рецепт вишневой наливки,"{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар': '30...","{'Вишня': '1 кг', 'Водка': '1 л', 'Сахар-песок..."
...,...,...,...,...
146576,https://www.povarenok.ru/recipes/show/61738/,"Украшение для блюд ""Снежинка""","{'Капуста краснокочанная': None, 'Капуста бело...","{'Капуста краснокочанная': None, 'Капуста бело..."
146577,https://www.povarenok.ru/recipes/show/83364/,Греческий рисовый пирог с фаршем,"{'Фарш мясной': '400 г', 'Масло сливочное': '2...","{'Фарш': '400 г', 'Масло сливочное': '2 ст. л...."
146578,https://www.povarenok.ru/recipes/show/80590/,Соус на груздях с хреном,"{'Грибы': '300 г', 'Чеснок': '5 зуб.', 'Лук ре...","{'Грибы': '300 г', 'Чеснок': '5 зуб.', 'Лук ре..."
146579,https://www.povarenok.ru/recipes/show/167099/,Печенье со сливочным сыром,"{'Масло сливочное': '225 г', 'Сыр сливочный': ...","{'Масло сливочное': '225 г', 'Сыр сливочный': ..."


### Стоп слова (вещи которые уже итак есть дома)

In [ ]:
stop_words_drop = [
    "Соль",
    "Сахар-песок",
    "Перец черный молотый",
    "Мука пшеничная",
    "Сода",
    "Сода гашеная уксусом",
]

In [3]:
import ast

all_ingredients = set()
for ingredients_str in data["ingredients_normalized"]:
    ingredients_parsed = ast.literal_eval(ingredients_str)
    all_ingredients.update(ingredients_parsed.keys())

all_ingredients = sorted(list(all_ingredients))
ingredient_to_idx = {ing: idx for idx, ing in enumerate(all_ingredients)}

print(f"Ингредиентов: {len(all_ingredients)}")
print(f"Рецептов: {len(data)}")

Ингредиентов: 979
Рецептов: 146581


In [ ]:
from tqdm import tqdm

interactions = []
for idx, row in tqdm(data.iterrows(), total=len(data)):
    ingredients_parsed = ast.literal_eval(row["ingredients_normalized"])
    recipe_id = row.get("url", idx)
    name = row.get("name", idx)
    for ingredient in ingredients_parsed.keys():
        interactions.append((recipe_id, name, ingredient))

interactions_df = pd.DataFrame(
    interactions, columns=["recipe_id", "name", "ingredient_id"]
)
print(f"Interactions {len(interactions_df)}")

unique_recipes = interactions_df["recipe_id"].unique()
all_unique_ingredients = interactions_df["ingredient_id"].unique()
unique_ingredients = [
    ingredient
    for ingredient in all_unique_ingredients
    # if ingredient not in stop_words_drop
]

recipe2id = {recipe: i for i, recipe in enumerate(unique_recipes)}
item2id = {ingredient: i for i, ingredient in enumerate(unique_ingredients)}

id2recipe = {i: recipe for recipe, i in recipe2id.items()}
id2item = {i: ingredient for ingredient, i in item2id.items()}

interactions_df["user_id"] = interactions_df["recipe_id"].map(recipe2id)
interactions_df["item_id"] = interactions_df["ingredient_id"].map(item2id)
interactions_df.dropna(subset=["item_id"], inplace=True)
interactions_df["item_id"] = interactions_df["item_id"].astype(int)
interactions_df

100%|██████████| 146581/146581 [00:08<00:00, 16317.75it/s]


Interactions 1278324


,recipe_id,name,ingredient_id,user_id,item_id
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Молоко,0,0
1,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Клубника,0,1
3,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Сыр твердый,1,2
4,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Чеснок,1,3
5,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Яйцо куриное,1,4
...,...,...,...,...,...
1278319,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Ванилин,146562,61
1278320,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Сметана,146562,58
1278321,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Яблоко,146562,67
1278322,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Апельсин,146562,129


#### Определяем частотность ингридиентов

In [ ]:
freq_df = interactions_df["ingredient_id"].value_counts().reset_index()
freq_df.columns = ["ingredient_id", "frequency"]


In [ ]:
freq_df.to_csv("freq_df.csv", index=False)  # сохраняем df с частотностью

In [10]:
freq_df

,ingredient_id,frequency
0,Яйцо куриное,58239
1,Масло растительное,57561
2,Лук репчатый,42061
3,Масло сливочное,37301
4,Чеснок,31338
...,...,...
968,Кунжутные семечки,1
969,Тесто блинное,1
970,Горбуша в масле,1
971,Изюм без косточек,1


In [ ]:
interactions_df = interactions_df.merge(
    freq_df,
    on="ingredient_id",
    how="left",  # чтобы ничего не потерять
)


In [ ]:
interactions_df["frequency"].isna().sum()


np.int64(0)

In [13]:
interactions_df

,recipe_id,name,ingredient_id,user_id,item_id,frequency
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Молоко,0,0,22625
1,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Клубника,0,1,2075
2,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Сыр твердый,1,2,16325
3,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Чеснок,1,3,31338
4,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Яйцо куриное,1,4,58239
...,...,...,...,...,...,...
1056577,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Ванилин,146562,61,7290
1056578,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Сметана,146562,58,18327
1056579,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Яблоко,146562,67,8462
1056580,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Апельсин,146562,129,3186


#### Ставим трешхолд на кол-во появлений в рецептах 

Чтобы убрать ингридиенты типо хвост, шкура свиная и т.д.

In [ ]:
interactions_df_selected = interactions_df[interactions_df["frequency"] > 500]
interactions_df_selected

,recipe_id,name,ingredient_id,user_id,item_id,frequency
0,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Молоко,0,0,22625
1,https://www.povarenok.ru/recipes/show/164365/,Густой молочно-клубничный коктейль,Клубника,0,1,2075
2,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Сыр твердый,1,2,16325
3,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Чеснок,1,3,31338
4,https://www.povarenok.ru/recipes/show/1306/,Рулетики,Яйцо куриное,1,4,58239
...,...,...,...,...,...,...
1056577,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Ванилин,146562,61,7290
1056578,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Сметана,146562,58,18327
1056579,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Яблоко,146562,67,8462
1056580,https://www.povarenok.ru/recipes/show/23318/,"Пирог ""Весенний"" с яблоками и апельсинами",Апельсин,146562,129,3186


Рандомно смотрим что у нас треш в данных

In [23]:
interactions_df_selected.sample(10)

,recipe_id,name,ingredient_id,user_id,item_id,frequency
434211,https://www.povarenok.ru/recipes/show/98701/,Торт-суфле с чёрной смородиной,Ванилин,60241,61,7290
182545,https://www.povarenok.ru/recipes/show/18315/,"Яичница ""Хочется весны""",Сельдерей корневой,25294,323,743
925740,https://www.povarenok.ru/recipes/show/9391/,Шашлык из овощей,Баклажан,128361,9,4012
294772,https://www.povarenok.ru/recipes/show/47685/,Сырный рулет,Сыр твердый,40896,2,16325
680411,https://www.povarenok.ru/recipes/show/109125/,Картофель в сыворотке по-новому,Сыворотка,94339,243,531
651532,https://www.povarenok.ru/recipes/show/15199/,Биточки из крабовых палочек,Масло растительное,90326,18,57561
466633,https://www.povarenok.ru/recipes/show/97311/,"Котлеты из двух видов рыб с соусом ""Луи""",Яйцо куриное,64715,4,58239
370858,https://www.povarenok.ru/recipes/show/51308/,"Карп на яблоках ""Праздничный""",Сметана,51502,58,18327
405964,https://www.povarenok.ru/recipes/show/31123/,Сливовый торт с безе,Миндаль,56371,88,2963
1028036,https://www.povarenok.ru/recipes/show/156007/,"Пирог с яблоками ""Русские узоры""",Разрыхлитель теста,142594,65,14793


+- адекватные ингридиенты